# Execute Commands on a Node

This notebook will show you execute commands on a FABRIC node.

## Setup the Experiment

#### Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Create the Slice

In [ ]:
try:
    slice_name="MySlice"
    node1_name="Node1"
    node2_name="Node2"

    
    # Create a slice
    slice = fablib.new_slice(name=slice_name)

    # Add a node
    slice.add_node(name=node1_name)
    slice.add_node(name=node2_name)

    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Get the Nodes

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.show()
    
    node1 = slice.get_node(name=node1_name)
    node1.show()
    
    node2 = slice.get_node(name=node2_name)
    node2.show()
except Exception as e:
    print(f"Exception: {e}")

## SSH from a Terminal Command

Connecting to your VMs requests you to jump through the bastion host to the VM using the keys configured above. From a terminal you can ssh to the VM using the command generated by the following command.  Note that you will need to replace the `-F` argument with the path to the ssh config file.  If you are using the default configuration in FABRIC JupyterHub, that path will be `/home/fabric/work/fabric_config/ssh_config`


In [ ]:
try:
    print(f"SSH Command: {node1.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

## Execute commands using FABlib

Most experiments will require automated configuration and execution. You can use the FABlib library to execute arbitrary commands on your VMs. 

The following cells use FABlib to execute a "Hello, FABRIC" bash script. The cell uses the bastion and VM keys defined in your fabric_rc file to jump through the bastion host and execute the script.

### Basic Execute

The `node.execute` command will run a shell command on your VM.  The resulting `stdout` and `stdin` will be returned as a tuple and printed as it is created.  

The command must be non-interactive.

In [ ]:
try:
    command = 'ping -c 10 www.google.com'

    stdout, stderr = node1.execute(command)
except Exception as e:
    print(f"Exception: {e}")

### Using Returned stdout and stderr

Set `quiet=True` to suppress printing.

In [ ]:
try:
    command = 'ping -c 10 www.google.com'

    stdout, stderr = node1.execute(command, quiet=True)
    
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")

except Exception as e:
    print(f"Exception: {e}")

### Write Output to a File

Set the `output_file` parameter to a file name and the output will be appended to the file as it is created.  

In this example, the file is located in the same folder as this notebook. This is useful for logging the configuration of a node.

In [ ]:
try:
    command = 'ping -c 10 www.google.com'

    stdout, stderr = node1.execute(command, output_file='node1.log')

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
!cat node1.log

## Execute the Command in a Thread

The `node.execute_thread` method will execute a command in a thread. When the command completes, the stdout and stderr tuple is returned by a blocking call to  `thread.result()`.  

Threaded execution is useful for configuring multiple FABRIC nodes in parallel.  Setting a unique file for each parallel configuration thread is The `output_file` is particularly useful for watching the progress of configuration running in parallel.



In [ ]:
try:
    command = 'ping -c 10 www.google.com'

    node1_thread = node1.execute_thread(command, output_file='node1.log')
    node2_thread = node2.execute_thread(command, output_file='node2.log')
    
    stdout1, stderr1 = node1_thread.result()
    stdout2, stderr2 = node2_thread.result()

    print(f"stdout1: {stdout1}")
    print(f"stderr1: {stderr1}")
    print(f"stdout2: {stdout2}")
    print(f"stderr2: {stderr2}")
    
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
!cat node1.log

!cat node2.log